# AiiDA's QueryBuilder

*Import statements - make sure to execute the cell below this one (it may be hidden)*

In [1]:
from datetime import datetime, timedelta
from aiida import load_dbenv, is_dbenv_loaded
if not is_dbenv_loaded():
    load_dbenv()
from aiida.orm import load_node, Node, Group, Computer, User
from aiida.orm import CalculationFactory, DataFactory
from aiida.orm.calculation.job import JobCalculation

PwCalculation = CalculationFactory('quantumespresso.pw')
StructureData = DataFactory('structure')
KpointsData = DataFactory('array.kpoints')
ParameterData = DataFactory('parameter')
UpfData = DataFactory('upf')

### 1. Introduction to the QueryBuilder
As you will use `AiiDA` to run your calculations, the database that stores all the data and the provenance, will quickly grow to be very large. So in order to find a needle that we might be looking for in this big haystack, we need an efficient excavation and search tool. The tool that `AiiDA` provides to do exactly this is called the `QueryBuilder`. The `QueryBuilder` acts as the gatekeeper to your database, that you can ask questions about the contents of your database (also referred to as queries), by specifying what it is that you are looking for. In this part of the tutorial, we will focus on how to use the `QueryBuilder` to make these queries and understand/use the results.

In order to use the `QueryBuilder`, we first need to import it. We can accomplish this by executing the `import` statement in the following cell. Go ahead and select the next cell, and press `Shift+Enter`.

In [2]:
from aiida.orm.querybuilder import QueryBuilder

Before we can ask the `QueryBuilder` questions about our database, we first need to create an instance of it:

In [3]:
qb = QueryBuilder()

Now that we have an instance of our `QueryBuilder` which we named `qb`, we are ready to start asking it about the contents of our database. For example, we may want to know exactly how many nodes there are in our database. To tell `qb` that we are interested in all the occurrences of the `Node` class in our database, we `append` it to the list of objects it should find.

In [4]:
qb.append(Node)

We have now narrowed down the scope of `qb` to just the nodes that are present in our database. To learn how many nodes there are exactly, all we have to do is to ask `qb` to count them.

In [5]:
qb.count()

320L

Now as you may have learned in previous sections of the tutorial, nodes come in different kinds and flavors. For example, all our crystal structures that we have stored in the database, are saved in a node that is of the type `StructureData`. If instead of all the nodes, rather we would like to count only the structure nodes, we simply tell our `QueryBuilder` to narrow its scope to only objects of the `StructureData` type. Since we are creating a new query, we have to create a new instance of the `QueryBuilder`. In the next cell, we have typed part of the code to count all the structure nodes. See if you can finish the line with the comment, to tell the `QueryBuilder` that you are only interested in `StructureData` nodes.

In [6]:
qb = QueryBuilder()
qb.append(StructureData) # How do we finish this line to tell the query builder to count only the structure nodes?
qb.count()

30L

Instead of just counting how many structure nodes we have, we may also actually want to see some of them. This is as easy as telling our `QueryBuilder` that we are not interested in the `count` but rather that we want to retrieve `all` the nodes.

In [7]:
qb = QueryBuilder()
qb.append(StructureData)
qb.all()

[[<StructureData: uuid: d8a39f61-8275-449b-a73b-da9713bf5354 (pk: 19)>],
 [<StructureData: uuid: 85bc8662-8a6e-4da1-946f-de04bea4e04b (pk: 23)>],
 [<StructureData: uuid: 79cbf093-fbbe-4465-9be0-a641767bb0ea (pk: 26)>],
 [<StructureData: uuid: 3c905b00-a1f8-4461-83fc-69ed660dee6b (pk: 33)>],
 [<StructureData: uuid: fa2e5d45-b4ac-4760-aa90-c8f0f4064fe1 (pk: 48)>],
 [<StructureData: uuid: 30e4a3b9-fdad-4a17-851f-68234facb34f (pk: 58)>],
 [<StructureData: uuid: 7493134d-54cd-406b-b39d-4ef99535e19b (pk: 63)>],
 [<StructureData: uuid: cfbc8e77-ace1-4cba-8519-2080addbf3c5 (pk: 65)>],
 [<StructureData: uuid: de8b2cd2-ed88-43b7-a201-9ef5d3bba7d8 (pk: 69)>],
 [<StructureData: uuid: 7a44f26c-22f1-4f36-b9c5-6da1103c4207 (pk: 72)>],
 [<StructureData: uuid: 7dd160bf-362c-4a79-91a3-a424f72a6e03 (pk: 75)>],
 [<StructureData: uuid: 1c9cf841-966a-446e-a35f-c7d2cb1fa00d (pk: 90)>],
 [<StructureData: uuid: 6c68fdb8-1092-4a90-8778-808ae39df63f (pk: 94)>],
 [<StructureData: uuid: 2e57053b-c04e-47b1-8cec-0ea

Note that this command is very literal and does in fact retrieve **all** the structure nodes that are stored in your database, which may become very slow if your database becomes very large. One solution is to simply tell the `QueryBuilder` that we are for example only interested in 5 structure nodes. This can be done with the `limit` method as follows:

In [8]:
qb = QueryBuilder()
qb.append(StructureData)
qb.limit(5)
qb.all()

[[<StructureData: uuid: d8a39f61-8275-449b-a73b-da9713bf5354 (pk: 19)>],
 [<StructureData: uuid: 85bc8662-8a6e-4da1-946f-de04bea4e04b (pk: 23)>],
 [<StructureData: uuid: 79cbf093-fbbe-4465-9be0-a641767bb0ea (pk: 26)>],
 [<StructureData: uuid: 3c905b00-a1f8-4461-83fc-69ed660dee6b (pk: 33)>],
 [<StructureData: uuid: fa2e5d45-b4ac-4760-aa90-c8f0f4064fe1 (pk: 48)>]]

Another option is to simply use the concept of array slicing, native to python, to specify a subset of the total return set to be returned

In [12]:
qb.limit(None)
qb.all()[:7]

[[<StructureData: uuid: d8a39f61-8275-449b-a73b-da9713bf5354 (pk: 19)>],
 [<StructureData: uuid: 85bc8662-8a6e-4da1-946f-de04bea4e04b (pk: 23)>],
 [<StructureData: uuid: 79cbf093-fbbe-4465-9be0-a641767bb0ea (pk: 26)>],
 [<StructureData: uuid: 3c905b00-a1f8-4461-83fc-69ed660dee6b (pk: 33)>],
 [<StructureData: uuid: fa2e5d45-b4ac-4760-aa90-c8f0f4064fe1 (pk: 48)>],
 [<StructureData: uuid: 30e4a3b9-fdad-4a17-851f-68234facb34f (pk: 58)>],
 [<StructureData: uuid: 7493134d-54cd-406b-b39d-4ef99535e19b (pk: 63)>]]

If we want to know a little bit more about the retrieved structure nodes, we can ask the `QueryBuilder` to return an iterator which we can then use in a straightforward `for`-loop. This allows one for example to print the formula of the structures

In [17]:
qb = QueryBuilder()
qb.append(StructureData)
qb.limit(5)
for structure, in qb.iterall():
    print structure.get_formula()

BaO3Ti
BaO3Ti
BaO3Ti
BaO3Ti
BaO3Ti


This is just a simple example how we can employ the `QueryBuilder` to get details about the contents of our database. We have now seen simple queries for the `Node` and `StructureData` classes of nodes, but the same rules apply to all the `AiiDA` node classes. For example we may want to count the number of entries for each of the node classes in the following list

In [18]:
class_list = [Node, StructureData, KpointsData, ParameterData, UpfData]

Using the tools we have learned so far, we can build a table of the number of occurrences of each of these node classes that are stored in our database. We simply loop over the `class_list` and create a `QueryBuilder` for each and count the entries.

In [19]:
for class_name in class_list:
    qb = QueryBuilder()
    qb.append(class_name)
    print '{:>15} | {:6}'.format(class_name.__name__, qb.count())

           Node |    320
  StructureData |     30
    KpointsData |      0
  ParameterData |     90
        UpfData |     79


If all went well, you should see something like the following, where of course the numbers may differ for your database

Class name     | Entries
---------------|--------
 Node          | 10273 
 StructureData | 271   
 KpointsData   | 953   
 ParameterData | 2922  
 UpfData       | 85    

### 2. Projection and filters
Up till now we have always asked the `QueryBuilder` to return the entire node, of whatever class of nodes that we specified. However, we might not necessarily be interested in all the node's properties, but rather just a select set or even just a single property. We can tell the `QueryBuilder` which properties we would like returned, by asking it to **project** those properties in the result. For example, we may only want to get the uuid's of a set of nodes. 

In [20]:
qb = QueryBuilder()
qb.append(Node, project=['uuid'])
qb.limit(5)
qb.all()

[[u'6220f6b9-1cfa-4ebc-ad79-aac46b48650d'],
 [u'6de1ab89-a7f0-4a36-b1b2-71e91c240f87'],
 [u'f4d608d3-59b7-43cd-b219-52a30baf8d51'],
 [u'0b6860d7-997d-4e9a-b3cb-50855c92f0d5'],
 [u'76617ab8-3359-41d1-8897-9832a0f11bd6']]

We inform the `QueryBuilder` that we are only interested in the `uuid` property of the `Node` class by specifing the `project` keyword in the `append` call. Note that the value that we assign to `project` is a list, as we may want to specify more than one property. See if you can get the `QueryBuilder` to return *both* the `id` and the `uuid` of the first 5 `Node`'s in the following cell.

In [21]:
qb = QueryBuilder()
qb.append(Node, project=['id', 'uuid'])#? What should the value be for the project key
qb.limit(5)
qb.all()

[[1, u'6220f6b9-1cfa-4ebc-ad79-aac46b48650d'],
 [2, u'6de1ab89-a7f0-4a36-b1b2-71e91c240f87'],
 [3, u'f4d608d3-59b7-43cd-b219-52a30baf8d51'],
 [4, u'0b6860d7-997d-4e9a-b3cb-50855c92f0d5'],
 [5, u'76617ab8-3359-41d1-8897-9832a0f11bd6']]

To give you an idea of the various properties that you can project for some of the `AiiDA` node classes you can consult the following table.
Note that this is by no means an exhaustive list

Class    | Properties
---------|-----------
Node     | `id`, `uuid`, `type`, `label`, `description`, `ctime`, `mtime`
Computer | `id`, `uuid`, `name`, `hostname`, `description`, `enabled`, `transport_type`, `scheduler_type`
User     | `id`, `email`, `first_name`, `last_name`, `institution`
Group    | `id`, `uuid`, `name`, `type`, `time`, `description`

The same properties can also be used to *filter* for specific nodes in your database. Up till now, we only ever asked to return the `QueryBuilder` all the instances of a certain type of node, or at best a limited number of those. But we might be interested in a very specific node. For example, we may have the `id` of a certain node and we would like to know when it was created and last modified. We can tell the `QueryBuilder` to select nodes that only match that criterion, by telling it to **filter** based on that property.

In [22]:
qb = QueryBuilder()
qb.append(Node, project=['ctime', 'mtime'], filters={'id': {'==': 1}})
qb.all()

[[datetime.datetime(2014, 4, 1, 14, 50, 55, 236925, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)),
  datetime.datetime(2017, 5, 12, 10, 45, 4, 851309, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None))]]

Note the syntax of the `filters` keyword. The value is a dictionary, where the keys indicate the node property that it is supposed to operate on, in this case the `id` property. The value of that key is again itself a dictionary, where the key indicates the logical operator `==` and the value corresponds to the value of the property.

You may have multiple criteria that you want to filter for, in which case you can use the logical `or` and `and` operators. Let's say, for example, that you want the `QueryBuilder` to retrieve all the `StructureData` nodes that have a certain `label` **and** where created no longer than 10 days ago. You can translate this criterion by making use of the `and` operator which allows you to specify multiple filters that all have to be satisfied.

In [23]:
qb = QueryBuilder()
qb.append(
    Node, 
    filters={
        'and': [
            {'ctime': {'>': datetime.now() - timedelta(days=12)}},
            {'label': 'graphene'}
        ]
    }
)
qb.all()

[]

You will have noticed that the `>` operator, and its related operators, can work with python `datetime` objects. These are just a few of the operators that `QueryBuilder` understands. Below you find a table with the various logical operators that you can use

Operator             | Data type             | Example                            | Description
---------------------|-----------------------|------------------------------------|------------------
`==`                 | all                   | `{'==': '12'}`                     | equality operator
`in`                 | all                   | `{'in':['FINISHED', 'PARSING']}`   | member of a set
`<`, `>`, `<=`, `>=` | float, int, datetime  | `{'>': 5.2}`                       | size comparison operator
`like`               | char, str             | `{'like': 'calculation%'}`         | string comparison, `%` is wildcard
`ilike`              | char, str             | `{'ilike': 'caLCulAtion%'}`        | string comparison, capital insensitive
`or`                 |                       | `{'or': [{'<': 5.3}, {'>': 6.3}]}` | logical or operator
`and`                |                       | `{'and': [{'>=': 2}, {'<=': 6}]}`  | logical and operator

As an exercise, try to write a query below that will retrieve all `Group` nodes whose `name` property starts with the string `tutorial`.

In [29]:
# Write your query here
qb = QueryBuilder()
qb.append(Group, filters={'name': {'like': 'tutorial%'}})
qb.limit(5)
qb.all()

[]

### 3. Defining relationships between query clauses

So far we have seen how we can use the `QueryBuilder` to search the database for entries of a specific node type, potentially projecting only specific properties and filtering for certain property values. However, our nodes do not live in a vacuum, but they are often related to one another. Especially, there will be interesting relationships between different types of nodes. Therefore we would like to be able to search for nodes, based on a certain relationship that they might have with other nodes. Consider for example that you have a `StructureData` node that was produced by some calculation. How would we be able to retrieve that calculation?

To accomplish this, we need to be able to tell the `QueryBuilder` what the relationship is between the nodes that we are interested in. If we take the example of the structure node but abstracting it to all structure nodes, we could do the following to find all the structure nodes that were generated as an output by a `PwCalculation` node.

In [30]:
qb = QueryBuilder()
qb.append(PwCalculation, tag='calculation')

We are looking for descendants of `PwCalculation` nodes, so we `append` it to a `QueryBuilder` instance. In the future, we need to be able to reference to this clause and therefore we give it a name or `tag` it with the `tag` keyword.

In [31]:
qb.append(StructureData, output_of='calculation')

The goal was to find `StructureData` nodes, so we `append` that to the `qb`. However, we didn't want to find just any `StructureData` nodes; they had to be an output of `PwCalculation`. Note how we expressed this relation by the `output_of` keyword and using the `tag` name `calculation` that we had just assigned in the previous `append` statement.
Now all we have to do is execute the query to retrieve our structures:

In [32]:
qb.limit(5)
qb.all()

[]

The `output_of` keyword is but one of many potential relationships that exist between the various `AiiDA` nodes, that are implemented in the `QueryBuilder`. The table below gives an overview of the implemented relationships, which nodes they are defined for and which relation it implicates.

Entity from	| Entity to	| Relationship  | Explanation
------------|-----------|---------------|------------
Node        | Node      | input_of      | One node as input of another node
Node        | Node      | output_of     | One node as output of another node
Node        | Node      | ancestor_of   | One node as the ancestor of another node
Node        | Node      | descendant_of | One node as descendant of another node
Group       | Node      | group_of      | The group of a node
Node        | Group     | member_of     | The node is a member of a group
Computer    | Node      | computer_of   | The computer of a node
Node        | Computer  | has_computer  | The node of a computer
User        | Node      | creator_of    | The creator of a node is a user
Node        | User      | created_by    | The node was created by a user

As an exercise, see if you can write a query that will return all the `UpfData` nodes that are a member of a `Group` whose name starts with the string `SSSP`.

In [ ]:
qb = QueryBuilder()
qb.append(Group, filters={'name': {'like': 'SSSP%'}}, tag='group')
qb.append(UpfData, member_of='group')
qb.all()[:5]

### 4. Attributes and extras

In section 2, we showed you how you can `project` specific properties of a `Node` and gave a list of properties that a `Node` instance possesses. Since then, we have come across a lot of different `AiiDA` data nodes, such as `StructureData` and `UpfData`, that were secretly `Node`'s in disguise. Or to put it correctly, as `AiiDA` employs the object-oriented programming paradigm, both `StructureData` and `UpfData` are examples of subclasses of the `Node` class and therefore inherit its properties. That means that whatever property a `Node` has, both `StructureData` and `UpfData` will have too. However, there is a semantic difference between a `StructureData` node and a `UpfData`, and so we may want to add a property to one that would not make sense for the other. To solve this, `AiiDA` introduces the concept of `attributes`. These are similar to properties, except that they are specific to the `Node` type that they are attached to. This allows you to add an `attribute` to a certain node, without having to change the implementation of all the others.

For example, the `ParameterData` nodes that are generated as output of `PwCalculation`'s may have an attribute named `wfc_cutoff`. To project for this particular `attribute`, one can use exactly the same syntax as shown in section 2 for the regular `Node` properties, and one has to only prepend `attributes.` to the attribute name. Demonstration:

In [ ]:
qb = QueryBuilder()
qb.append(PwCalculation, tag='pw')
qb.append(
    ParameterData,
    output_of='pw',
    project=["attributes.wfc_cutoff"]
)
qb.all()[:5]

Note that not every `ParameterData` node has to have this attribute, in which case the `QueryBuilder` will return the python keyword `None`. Similar to the `attributes`, nodes also can have `extras`, which work in the same way, except that `extras` are mutable, which means that their value can be changed even after a node instance has been stored.

If you are not sure which attributes a given node has, you can use the `get_attrs()` method to simply retrieve them all. It will return a dictionary with all the attributes the node has.

In [ ]:
qb = QueryBuilder()
qb.append(PwCalculation)
node = qb.first()[0]
node.get_attrs()

The element type symbol of a pseudopotential, that is represented by a `UpfData` node, is stored in the `element` attribute. Using the knowledge that filtering on attributes works exactly as for normal node properties, see if you can write a query that will search your database for pseudopotentials for silicon.

In [ ]:
qb = QueryBuilder()
qb.append(UpfData, filters={'attributes.element': {'==': 'Si'}})
qb.all()